In [3]:
import os

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

tf.version.VERSION

'2.4.0'

In [2]:
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()

y_train = y_train[:1000]
y_test = y_test[:1000]
X_train = X_train[:1000].reshape(-1, 28*28) / 255.
X_test = X_test[:1000].reshape(-1, 28*28) / 255.

In [4]:
def create_mod():
    mod = Sequential([Dense(512, activation='relu', input_shape=(784,)),
                      Dropout(0.2),
                      Dense(10)])
    mod.compile(
        optimizer='adam',
        loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[tf.metrics.SparseCategoricalAccuracy()])
    return mod

In [5]:
mod = create_mod()
mod.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


### Save checkpoints during training

In [6]:
checkpoint_path = 'training_1/cp.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, save_weights_only=True, verbose=1)
mod.fit(X_train,
        y_train,
        epochs=10,
        validation_data=(X_test, y_test),
        callbacks=[cp_callback])

Epoch 1/10
32/32 [==============================] - 1s 21ms/step - loss: 1.5487 - sparse_categorical_accuracy: 0.5292 - val_loss: 0.6819 - val_sparse_categorical_accuracy: 0.8000

Epoch 00001: saving model to training_1/cp.ckpt
Epoch 2/10
32/32 [==============================] - 0s 4ms/step - loss: 0.4047 - sparse_categorical_accuracy: 0.8917 - val_loss: 0.4997 - val_sparse_categorical_accuracy: 0.8450

Epoch 00002: saving model to training_1/cp.ckpt
Epoch 3/10
32/32 [==============================] - 0s 4ms/step - loss: 0.3252 - sparse_categorical_accuracy: 0.9156 - val_loss: 0.4511 - val_sparse_categorical_accuracy: 0.8570

Epoch 00003: saving model to training_1/cp.ckpt
Epoch 4/10
32/32 [==============================] - 0s 4ms/step - loss: 0.2030 - sparse_categorical_accuracy: 0.9507 - val_loss: 0.4206 - val_sparse_categorical_accuracy: 0.8630

Epoch 00004: saving model to training_1/cp.ckpt
Epoch 5/10
32/32 [==============================] - 0s 4ms/step - loss: 0.1401 - sparse_cat

In [7]:
os.listdir(checkpoint_dir)

['cp.ckpt.data-00000-of-00001', 'checkpoint', 'cp.ckpt.index']

In [9]:
mod = create_mod()
loss, acc = mod.evaluate(X_test, y_test, verbose=2)
print(f'Untrained: Acc: {100 * acc:5.2f}')

32/32 - 0s - loss: 2.3442 - sparse_categorical_accuracy: 0.1360
Untrained: Acc: 13.60


In [10]:
mod.load_weights(checkpoint_path)
loss, acc = mod.evaluate(X_test, y_test, verbose=2)
print(f'Loaded weights: Acc: {100 * acc:5.2f}')

32/32 - 0s - loss: 0.4284 - sparse_categorical_accuracy: 0.8630
Loaded weights: Acc: 86.30


### Checkpoint Callback Options